In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys

module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from generator import RoadNetwork
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import pandas as pd
import networkx as nx
import numpy as np
from tqdm import tqdm

from models import Node2VecModel

In [2]:
city = "sf"
city_traj = "SF"

In [3]:
network = RoadNetwork()
network.load(f"../../osm_data/{city}")

In [8]:
data = network.generate_road_segment_pyg_dataset(traj_data=None, include_coords=True, dataset=city)

In [6]:
print(torch.cuda.is_available())
print(torch.cuda.current_device())
print(torch.cuda.device_count())
torch.cuda.set_device(1)
print(torch.cuda.current_device())
print(torch.cuda.device_count())

True
1
4
1
4


In [13]:
from torch_geometric.data import Data
import torch_geometric.transforms as T

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
transform = T.Compose([
    T.ToDevice(device),
])
data = transform(data)
model = Node2VecModel(data, device=device, q=4, p=1)
model.train(epochs=20)

In [14]:
model.save_model(path="../model_states/node2vec/")
# model.save_emb(path="../model_states/node2vec/")

In [11]:
from sklearn import model_selection
from sklearn import linear_model
from sklearn import metrics
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

X = model.load_emb() # embedding for each node
# train simple classifier on 80% of data with cross validation
y = np.array([network.gdf_edges.loc[n]["highway_enc"] for n in network.line_graph.nodes])

#mask = ((y==11) | (y==10) | (y==9) | (y==4) | (y==1) | (y==2) | (y==12) | (y==7)) # remove uncommon tags
#X = X[~mask, :]
#y = y[~mask]
print(np.unique(y, return_counts=True))

X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size= 0.2, random_state = 1)

print('X_train dimension= ', X_train.shape)
print('X_test dimension= ', X_test.shape)
print('y_train dimension= ', y_train.shape)
print('y_test dimension= ', y_test.shape)

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13]), array([   92,    18,   127,   150,  1357,   108, 17847,  2653,    86,
        3737,    31,   228,    33,   572]))
X_train dimension=  (21631, 128)
X_test dimension=  (5408, 128)
y_train dimension=  (21631,)
y_test dimension=  (5408,)


In [12]:
lm = linear_model.LogisticRegression(multi_class="multinomial", max_iter=1000)
lm.fit(X_train, y_train)
print(metrics.classification_report(y_test, lm.predict(X_test)))

              precision    recall  f1-score   support

           0       0.25      0.19      0.21        16
           1       0.00      0.00      0.00         8
           2       0.56      0.43      0.49        23
           3       0.67      0.29      0.40        35
           4       0.35      0.11      0.17       246
           5       0.00      0.00      0.00        16
           6       0.71      0.96      0.82      3574
           7       0.40      0.18      0.25       515
           8       0.00      0.00      0.00        19
           9       0.37      0.08      0.14       779
          10       0.00      0.00      0.00         4
          11       0.29      0.04      0.07        53
          12       0.00      0.00      0.00        10
          13       0.28      0.14      0.18       110

    accuracy                           0.68      5408
   macro avg       0.28      0.17      0.19      5408
weighted avg       0.59      0.68      0.60      5408

